In [1]:
import pandas as pd
import spacy
from tqdm import tqdm

In [2]:
non_linked = pd.read_csv('csv_lifting/data/dataset.csv')

In [3]:
disease = spacy.load("en_core_sci_sm")
disease.add_pipe('entityfishing', config={
        "extra_info": True,
        "api_ef_base": "http://nerd.huma-num.fr/nerd/service"
    })

In [4]:
non_linked.columns

Index(['Disease', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4',
       'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9',
       'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14',
       'Symptom_15', 'Symptom_16', 'Symptom_17'],
      dtype='object')

In [5]:
non_linked[:10]

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
disease_linked = []

for item in tqdm(non_linked['Disease']):
    doc = disease(item)
    found = False
    for ent in doc.ents:
        if len(ent) == len(doc):
            disease_linked.append(ent._.kb_qid)
            found = True
            break
    if not found:
        disease_linked.append(None)

 80%|████████  | 3960/4920 [05:51<01:25, 11.26it/s]


KeyboardInterrupt: 

In [7]:
len(disease_linked), len(non_linked['Disease'])

(3960, 4920)

In [8]:
d1 = pd.DataFrame({
    'link_disease': disease_linked
})
d1.to_csv('diseases.csv')

In [9]:
non_linked['Symptom_5'][0]

nan

In [10]:
linked_symptoms = []

for i in range(17):
    linked_symptoms.append([])

for line in tqdm(range(len(non_linked))):
    for i in range(1, 18):
        column_name = f"Symptom_{str(i)}"
        if str(non_linked[column_name][line]) == "nan" or non_linked[column_name][line] is None:
            linked_symptoms[i - 1].append(None)
            continue
        doc = disease(non_linked[column_name][line].replace("_", " "))
        found = False
        for ent in doc.ents:
            if len(ent) == len(doc):
                linked_symptoms[i - 1].append(ent._.kb_qid)
                found = True
                break
        if not found:
            linked_symptoms[i - 1].append(None)

100%|██████████| 4920/4920 [45:50<00:00,  1.79it/s]  


In [11]:
[len(s) for s in linked_symptoms]

[4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920,
 4920]

In [14]:
d2 = pd.DataFrame({
    f"linked_symptom_{str(i+1)}": linked_symptoms[i] for i in range(len(linked_symptoms))
})
d2.to_csv('linked_symptoms.csv')